## Processing and uploading data to Synapse

In [ ]:
import synapseclient
syn = synapseclient.Synapse()
syn.login()

In [1]:
!cd /home/kipkurui/Dream_challenge/annotations/

In [ ]:
!cd /home/kipkurui/Dream_challenge/annotations/

a = """TF Name	Training Cell Types	Leaderboard Cell Types	Final Submission Cell Types
ARID3A	HepG2	K562	
ATF2	GM12878, H1-hESC, MCF-7	K562	HepG2
ATF3	HCT116, H1-hESC, HepG2, K562	liver	
ATF7	GM12878, HepG2, K562	MCF-7	
CEBPB	A549, H1-hESC, HCT116, HeLa-S3, HepG2, IMR-90, K562	MCF-7	
CREB1	GM12878, H1-hESC, HepG2, K562	MCF-7	
CTCF	A549, H1-hESC, HeLa-S3, HepG2, IMR-90, K562, MCF-7	GM12878	PC-3, induced_pluripotent_stem_cell
E2F1	GM12878, HeLa-S3		K562
E2F6	A549, H1-hESC, HeLa-S3	K562	
EGR1	GM12878, H1-hESC, HCT116, MCF-7	K562	liver
EP300	GM12878, H1-hESC, HeLa-S3, HepG2, K562, SK-N-SH	MCF-7	
FOXA1	HepG2	MCF-7	liver
FOXA2	HepG2		liver
GABPA	GM12878, H1-hESC, HeLa-S3, HepG2, MCF-7, SK-N-SH	K562	liver
GATA3	A549, SK-N-SH	MCF-7	
HNF4A	HepG2		liver
JUND	HCT116, HeLa-S3, HepG2, K562, MCF-7, SK-N-SH	H1-hESC	liver
MAFK	GM12878, H1-hESC, HeLa-S3, HepG2, IMR-90	K562, MCF-7	
MAX	A549, GM12878, H1-hESC, HCT116, HeLa-S3, HepG2, K562, SK-N-SH	MCF-7	liver
MYC	A549, HeLa-S3, K562, MCF-7	HepG2	
NANOG	H1-hESC		induced_pluripotent_stem_cell
REST	H1-hESC, HeLa-S3, HepG2, MCF-7, Panc1, SK-N-SH	K562	liver
RFX5	GM12878, HeLa-S3, MCF-7, SK-N-SH	HepG2	
SPI1	GM12878	K562	
SRF	GM12878, H1-hESC, HCT116, HepG2, K562	MCF-7	
STAT3	HeLa-S3	GM12878	
TAF1	GM12878, H1-hESC, HeLa-S3, K562, SK-N-SH	HepG2	liver
TCF12	GM12878, H1-hESC, MCF-7, SK-N-SH	K562	
TCF7L2	HCT116, HeLa-S3, Panc1	MCF-7	
TEAD4	A549, H1-hESC, HCT116, HepG2, K562, SK-N-SH	MCF-7	
YY1	GM12878, H1-hESC, HCT116, HepG2, SK-N-SH	K562	
ZNF143	GM12878, H1-hESC, HeLa-S3, HepG2	K562	""".split("\n")

TF_dict = {}
for i in a:
    line = i.split("\t")
    TF_dict[line[0]] = line[1:]
    

# for tfs in "FOXA1 EGR1".split():
#     cl = TF_dict[tfs][1].split(",")[0]
#     !paste --d='\t'  ladder_regions.blacklistfiltered.bed  ladder/{tfs}/{tfs}_xgb_v2.txt >ladder/{tfs}/L.{tfs}.{cl}.tab
#     !gzip ladder/{tfs}/L.{tfs}.{cl}.tab

### Get the Data from CHPC

In [ ]:
for tf in "EGR1 ATF2 FOXA1 FOXA2 HNF4A NANOG REST JUND MAX GABPA".split():
    !scp -r ckibet@lengau.chpc.ac.za:/home/ckibet/lustre/Dream_challenge/annotations/{tf} final/

### Convert the dat into a format ready for upload

In [ ]:
for tf in ["TAF1"]: #.split():
    tfs = "GABPA"
    cl = TF_dict[tf][2].split(",")[-1].strip()
    !paste --d='\t'  test_regions.blacklistfiltered.bed  final/{tfs}/{tfs}_xgb_final.txt >final/{tf}/F.{tf}.{cl}.tab
    !gzip final/{tf}/F.{tf}.{cl}.tab

### Upload the data to Synapse

In [ ]:
from synapseclient import Project

project = Project('TEAMKE_DREAM_submissions')
project = syn.store(project)

submissions_folder = "syn7317365"

from synapseclient import File, Activity

for tfs in ["TAF1"]: #.split():#"ATF2 EGR1 FOXA1 FOXA2 HNF4A NANOG REST JUND MAX".split():
    cl = TF_dict[tfs][2].split(",")[-1].strip()
    test_entity = File('final/%s/F.%s.%s.tab.gz' % (tfs,tfs,cl), description='Final submission', parent=submissions_folder)
    test_entity = syn.store(test_entity)

    evaluation = syn.getEvaluation(7212779)
    #entity = syn.get(test_entity.uri.split("/")[-1])
    submission = syn.submit(evaluation, test_entity, name='Final Submission', team='TeamKE')